In [11]:
import random
import math
from decimal import Decimal
import time

def isprime(n):
    if n == 2:
        return True
    if n == 1 or n % 2 == 0:
        return False
    i = 3
    while i <= math.sqrt(n):
        if n % i == 0:
            return False
        i = i + 2
    return True


def initial(Z_lower=100):
    start = time.time()
    # generate q bigger than z_lower
    q = Z_lower
    while True:
        if isprime(q):
            break
        else:
            q = q + 1
    print("q = " + str(q))
    print("\nq is prime\n")

    # Find p and r
    r = 1
    while True:
        p = r * q + 1
        if isprime(p):
            print("r = " + str(r))
            print("p = " + str(p))
            print("\np is prime\n")
            break
        r = r + 1

    # Compute elements of Z_p*
    Z_p_star = []
    for i in range(0, p):
        if (math.gcd(i, p) == 1):
            Z_p_star.append(i)
        # if len(Z_p_star) > 10:
        #     break

    # print("Z_p* = ")
    # print(Z_p_star) # , len(Z_p_star) same length, i.e. range(p)

    # Compute elements of G = {h^r mod p | h in Z_p*}
    G = []
    for i in Z_p_star:
        G.append((i ** r) % p)

    G = list(set(G))
    G.sort()
    # print("\nG = ")
    # print(G)
    print("Order of G is " + str(len(G)) + ". This must be equal to q:",q)

    # Since the order of G is prime, any element of G except 1 is a generator
    g = random.choice(list(filter(lambda g: g != 1, G)))
    print("\ng = " + str(g) + "\n")

    print("initialization time ", time.time()-start)
    return p, q, r, g

def generate_shares(n, t, secret, p, q, r, g, idxs):
    if secret==0: secret=1 # one exception after quantization
    assert secret >= 1 and secret <= q, "secret not in range "+str(secret)

    FIELD_SIZE = q
    coefficients = coeff(t, secret, FIELD_SIZE)
    # print("coefficients",coefficients,type(coefficients[0]))
    users = list(idxs) # users are to recieve the shares
    assert n==len(users), "these two number should be identical"

    shares = []
    for i in users:
        f_i = f(i, coefficients, q)
        shares.append((i, f_i))

#     start = time.time()
#     commitments = commitment(coefficients, g, p)
#     print("commitments:", commitments)
#     print("commitments take ", time.time()-start,"seconds")

#     verifications = []
#     startv = time.time()
#     for i in users:
#         # start = time.time()
#         check1 = quick_pow(g, share_ith(shares, i), p)
#         # print("check1 time ", time.time()-start)
#         # start = time.time()
#         # check1 = g ** share_ith(shares, i) % p
#         check2 = verification(g, commitments, i, p, q)
#         # print("check2 time ", time.time()-start)
#         verifications.append(check2)
#         if (check1 % p)  == (check2  % p) :
#             pass
#         else:
#             # print(g, share_ith(shares, i), p , q, i, commitments, shares)
#             print("checking fails with:", check1, check2)
#         # print(i, "-th user ============= tag at time ", time.time()-start,"seconds =============")
#     print("verification time ", time.time()-startv)
    commitments, verifications = [0,], [0,]
    return shares, commitments, verifications

def share_ith(shares, i):
    for share in shares:
        if share[0] == i:
            return int(share[1])
    return None

def coeff(t, secret, FIELD_SIZE):
    coeff = [random.randrange(0, FIELD_SIZE) for _ in range(t - 1)]
    coeff.append(secret)  # a0 is secret
    return coeff

def f(x, coefficients, q):
    # y = Decimal('0')
    # for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
    #     y += (Decimal(str(x)) ** Decimal(str(coefficient_index)) * Decimal(str(coefficient_value)))
    #     y = Decimal(int(y)%q)
    # return int(y)
    y = 0
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        y += x ** coefficient_index * coefficient_value
        y = int(y) % q
    return y

def commitment(coefficients, g, p):
    commitments = []
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        # c = g ** coefficient_value % p
        c = quick_pow(g, coefficient_value, p)
        commitments.append(c)
    return commitments

def verification(g, commitments, i, p, q):
    v = 1
    for k, c in enumerate(commitments):
        # start = time.time()
        # v = ( v * (c ** ((i ** k) % q)) ) % p
        v = (v * quick_pow(c, (i ** k) % q , p)) % p
        # print(c,i,k)
        # print("verification once takes:", time.time() - start)
    return v

def quick_pow(a, b, q):  # compute a^b mod q, in a faster way？
    temp = 1
    for i in range(1, b + 1):
        temp = (temp * a) % q
    return temp % q


def reconstruct_secret(pool, q):
    start = time.time()
    x_s,y_s = [],[]
    for share in pool:
        x_s.append(int(share[0]))
        y_s.append(int(share[1]))
    out = _lagrange_interpolate(0, x_s, y_s, q)
    print("reconstruct_secret time takes:", time.time() - start)
    return out

def _lagrange_interpolate(x, x_s, y_s, p):
    """
    Find the y-value for the given x, given n (x, y) points;
    k points will define a polynomial of up to kth order.
    """
    k = len(x_s)
    assert k == len(set(x_s)), "points must be distinct"

    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum *= v
        return accum

    nums = []  # avoid inexact division
    dens = []
    for i in range(k):
        others = list(x_s)
        cur = others.pop(i)
        nums.append(PI(int(x - o) for o in others))
        dens.append(PI(int(cur - o) for o in others))
    den = PI(dens)
    num = sum([_divmod(int(int(nums[i]) * int(den) * int(y_s[i]) % p), int(dens[i]), p) for i in range(k)])
    # debug: overflow,  ^ here  :  cast to int
    # num = 0
    # for i in range(k):
    #     temp = int(int(nums[i]) * int(den) * int(y_s[i]) % p)
    #     num += _divmod(temp, int(dens[i]), p)

    return (_divmod(num, den, p) + p) % p

def _extended_gcd(a, b):
    """
    Division in integers modulus p means finding the inverse of the
    denominator modulo p and then multiplying the numerator by this
    inverse (Note: inverse of A is B such that A*B % p == 1) this can
    be computed via extended Euclidean algorithm
    http://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation
    """
    x = 0
    last_x = 1
    y = 1
    last_y = 0
    while b != 0:
        quot = a // b
        a, b = b, a % b
        x, last_x = last_x - quot * x, x
        y, last_y = last_y - quot * y, y
    return int(last_x), int(last_y)

def _divmod(num, den, p):
    """Compute num / den modulo prime p

    To explain what this means, the return value will be such that
    the following is true: den * _divmod(num, den, p) % p == num
    """

    inv, _ = _extended_gcd(den, p)
    return num * inv

In [12]:
# initialization
time_start = time.time()
print("========Main VSS Starts==========")
p,q,r,g = initial(2* 10**8) #   2 * 10**7)

# Secret taken from the group Z_q* 
t, n = 7, 40
secret = q - 123
print(f'Original Secret: {secret}')

# Phase I: Generation of shares
users = [i+1 for i in range(n)]
# print(n, t, secret, p, q, r, g, users)
shares, commitments, verifications= generate_shares(n, t, secret, p, q, r, g, users)
print(f'Shares: {", ".join(str(share) for share in shares)}')
# print(f'Commitments: {", ".join(str(commitment) for commitment in commitments)}')
# print(f'verifications: {", ".join(str(verification) for verification in verifications)}')

# Phase II: Secret Reconstruction
# Picking t shares randomly for reconstruction
pool = random.sample(shares, t)
# print(f'Combining shares: {", ".join(str(share) for share in pool)}')
secret_reconstructed = reconstruct_secret(pool, q)
print("reconstruct_secret:",secret_reconstructed)

time_end = time.time()
print('time cost in second:', time_end-time_start)

========Main VSS Starts==========
q = 200000033

q is prime

r = 2
p = 400000067

p is prime

Order of G is 200000033. This must be equal to q: 200000033

g = 18867703

initialization time  490.7028305530548
Original Secret: 199999910
Shares: (1, 156535531), (2, 181419436), (3, 23114365), (4, 140592682), (5, 83228907), (6, 416678), (7, 80909285), (8, 121884139), (9, 27731308), (10, 38565922), (11, 88464348), (12, 93424432), (13, 169049643), (14, 177957020), (15, 6909153), (16, 173670132), (17, 163585035), (18, 95883879), (19, 119709110), (20, 139867061), (21, 72303411), (22, 29302546), (23, 34410756), (24, 67083400), (25, 37055973), (26, 88439141), (27, 33537546), (28, 116392744), (29, 106049781), (30, 119547968), (31, 174635360), (32, 72207203), (33, 8468415), (34, 116819804), (35, 139468221), (36, 28760780), (37, 78242914), (38, 183440201), (39, 32364257), (40, 25742663)
reconstruct_secret time takes: 0.00013685226440429688
reconstruct_secret: 199999910
time cost in second: 514.12604

In [13]:
k = 7
n = 40
p = q

F = FiniteField(p = p)
enc, dec, solveSystem = makeEncoderDecoder(n, k, p)

RS_reconstruct_secret(shares, F, solveSystem)

reconstruct_secret time takes: 0.1376042366027832


199999910

In [4]:
from finitefield.finitefield import FiniteField
from finitefield.polynomial import polynomialsOver
from welchberlekamp import makeEncoderDecoder
# https://jeremykun.com/2015/09/07/welch-berlekamp/
# https://github.com/j2kun/welch-berlekamp

In [5]:
# test1
p_number = q
F = FiniteField(p=p_number)
a = F(7)
print(1/a)
P = polynomialsOver(F)
g = P([1,3,5])
print(g)
print(g*g)
print(g(100))
print((1+ 3*100**1 + 5*100**2) % p_number )

1142859
1 + 3 t^1 + 5 t^2
1 + 6 t^1 + 19 t^2 + 30 t^3 + 25 t^4
50301
50301


In [6]:
# test2
integerMessage = [2,7,2,1]
k = 4
n = 7
p = 13
enc, dec, solveSystem = makeEncoderDecoder(n, k, p)
encoded = enc(integerMessage)
print("encoded message is: %r" % (encoded,)) # cleaner output
e=1
corrupted = encoded[:]
corrupted[0][1] = corrupted[0][1] + 1
print("corrupted message is: %r" % (corrupted,))
Q,E = solveSystem(corrupted)
P, remainder = (Q.__divmod__(E))

print("P(x) = %r" % P)
print(P.coefficients)

encoded message is: [[0 (mod 13), 2 (mod 13)], [1 (mod 13), 12 (mod 13)], [2 (mod 13), 6 (mod 13)], [3 (mod 13), 3 (mod 13)], [4 (mod 13), 9 (mod 13)], [5 (mod 13), 4 (mod 13)], [6 (mod 13), 7 (mod 13)]]
corrupted message is: [[0 (mod 13), 3 (mod 13)], [1 (mod 13), 12 (mod 13)], [2 (mod 13), 6 (mod 13)], [3 (mod 13), 3 (mod 13)], [4 (mod 13), 9 (mod 13)], [5 (mod 13), 4 (mod 13)], [6 (mod 13), 7 (mod 13)]]
P(x) = 2 + 7 t^1 + 2 t^2 + 1 t^3
[2 (mod 13), 7 (mod 13), 2 (mod 13), 1 (mod 13)]


In [8]:
import copy

k = 7
n = 40
p = q

F = FiniteField(p = p)
enc, dec, solveSystem = makeEncoderDecoder(n, k, p)

encoded_msg = []
for share in shares:
    x, y = share[0], share[1]
    encoded_msg.append( [F(x),F(y)] )

print(encoded_msg[0])
corrupted = copy.deepcopy(encoded_msg[:])
corrupted[0][1] = corrupted[0][1] + 1
print(corrupted[0])

Q,E = solveSystem(corrupted)
P, remainder = (Q.__divmod__(E))
print("P(x) = %r" % P)
print(P.coefficients)
print(P(0))

pool = random.sample(shares, k)
secret_reconstructed = reconstruct_secret(pool, q)
print(secret_reconstructed)

[1 (mod 2000003), 634483 (mod 2000003)]
[1 (mod 2000003), 634484 (mod 2000003)]
P(x) = 1999880 + 755540 t^1 + 123202 t^2 + 959581 t^3 + 573328 t^4 + 1016572 t^5 + 1206389 t^6
[1999880 (mod 2000003), 755540 (mod 2000003), 123202 (mod 2000003), 959581 (mod 2000003), 573328 (mod 2000003), 1016572 (mod 2000003), 1206389 (mod 2000003)]
1999880
reconstruct_secret time takes: 8.249282836914062e-05
1999880


In [9]:
def RS_reconstruct_secret(encoded_shares, Field, solveSystem):
    start = time.time()
    encoded_msg = []
    for share in encoded_shares:
        x, y = share[0], share[1]
        encoded_msg.append( [Field(x),Field(y)] )
    Q,E = solveSystem(encoded_msg)
    P, remainder = (Q.__divmod__(E))
    out = int(P(0))
    print("reconstruct_secret time takes:", time.time() - start)
    return out

RS_reconstruct_secret(shares, F, solveSystem)

reconstruct_secret time takes: 0.12392830848693848


1999880

In [ ]:
# LCC

In [20]:
import random
import math
# from decimal import Decimal
import time

def isprime(n):
    if n == 2:
        return True
    if n == 1 or n % 2 == 0:
        return False
    i = 3
    while i <= math.sqrt(n):
        if n % i == 0:
            return False
        i = i + 2
    return True


def initial(Z_lower=100):
    # generate q bigger than z_lower
    q = Z_lower
    while True:
        if isprime(q):
            break
        else:
            q = q + 1
    print("q = " + str(q))
    print("\nq is prime\n")

    # Find p and r
    r = 1
    while True:
        p = r * q + 1
        if isprime(p):
            print("r = " + str(r))
            print("p = " + str(p))
            print("\np is prime\n")
            break
        r = r + 1

    # Compute elements of Z_p*
    Z_p_star = []
    for i in range(0, p):
        if (math.gcd(i, p) == 1):
            Z_p_star.append(i)
        # if len(Z_p_star) > 1:
        #     break

    # print("Z_p* = ")
    # print(Z_p_star) # , len(Z_p_star) same length, i.e. range(p)

    # Compute elements of G = {h^r mod p | h in Z_p*}
    G = []
    for i in Z_p_star:
        G.append(i ** r % p)

    G = list(set(G))
    G.sort()
    # print("\nG = ")
    # print(G)
    # print("Order of G is " + str(len(G)) + ". This must be equal to q.")

    # Since the order of G is prime, any element of G except 1 is a generator
    g = random.choice(list(filter(lambda g: g != 1, G)))
    print("\ng = " + str(g) + "\n")

    return p, q, r, g


def generate_shares(N, T, K, secrets, p, q, r, g, alphas, betas):
    secrets_check = []
    for secret in secrets:
        if secret == 0:
            secrets_check.append(1)  # "0": one exception after quantization
        else:
            secrets_check.append(secret)
    secrets = secrets_check
    for secret in secrets:
        assert secret >= 1 and secret <= q, "secret not in range"

    FIELD_SIZE = q
    noises = [random.randrange(0, FIELD_SIZE) for _ in range(T)]

    shares = []
    for alpha in alphas:
        y = _lagrange_interpolate(alpha, betas, secrets + noises, q)
        shares.append((alpha, y))

    commitments = commitment(secrets + noises, g, p)
    start = time.time()

    verifications = []
    for alpha in alphas:
        # check1 = g ** shares[i-1][1] % p
        # check1 = g ** share_ith(shares, i) % p
        check1 = quick_pow(g, share_ith(shares, alpha), p)
        check2 = verification(commitments, alpha, betas, p, q)
        verifications.append(check2)
        if (check1 % p) == (check2 % p):
            pass
        else:
            print("checking fails with:", check1, check2)
            1/0
#         print(alpha, "-th user ============= tag at time ", time.time()-start,"seconds =============")
        start = time.time()
    # commitments, verifications = [0,], [0,]
    return shares, commitments, verifications


def share_ith(shares, i):
    for share in shares:
        if share[0] == i:
            return share[1]
    return None


def quick_pow(a, b, q):  # compute a^b mod q, in a faster way？
    temp = 1
    for i in range(1, b + 1):
        temp = (temp * a) % q
    return temp % q


def commitment(paras, g, p):
    commitments = []
    for para in paras:
        # c = g ** coefficient_value % p
        c = quick_pow(g, para, p)
        commitments.append(c)
    return commitments


def verification(commitments, alpha, betas, p, q):
    # v_pos, v_neg = 1, 1
    v = 1
    for i, c in enumerate(commitments):
        num, den = 1, 1
        for k, _ in enumerate(commitments):
            if k != i:
                num *= alpha - betas[k]
                den *= betas[i] - betas[k]
            else:
                pass
        # if num / den > 0:
        #     v_pos = v_pos * quick_pow(c, int(num / den) % q, p) # c ** int(num / den) % p
        # else:
        #     v_neg = v_neg * quick_pow(c, int(- num / den) % q, p) # c ** int(-num / den) % p

        # v = (v * quick_pow(c, int(num / den) % q, p)) % p
        v = (v * quick_pow(c, _divmod(num, den, q) % q , p)) % p
    # v = _divmod(v_pos, v_neg, p)
    return v


def reconstruct_secret(pool, q, betas, K):
    start =  time.time()
    out = []
    x_s, y_s = [], []
    for share in pool:
        x_s.append(int(share[0]))
        y_s.append(int(share[1]))
    for k in range(K):
        beta = betas[k]
        # out.append(f_rec(beta,pool,q))
        out.append(_lagrange_interpolate(beta, x_s, y_s, q))
    print("reconstruct_secret time takes:", time.time() - start)
    return out


def _lagrange_interpolate(x, x_s, y_s, q):
    """
    Find the y-value for the given x, given n (x, y) points;
    k points will define a polynomial of up to kth order.
    """
    k = len(x_s)
    assert k == len(set(x_s)), "points must be distinct"

    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum *= v
        return accum

    nums = []  # avoid inexact division
    dens = []
    L = 0
    for i in range(k):
        others = list(x_s)
        cur = others.pop(i)
        nums.append(PI(x - o for o in others))
        dens.append(PI(cur - o for o in others))

        L += _divmod(y_s[i] * nums[i], dens[i], q) 
    # den = PI(dens)
    # num = sum([_divmod(nums[i] * den * y_s[i] % q, dens[i], q) for i in range(k)])

    # L = sum( [_divmod(y_s[i] * nums[i], dens[i], q) for i in range(k)] )

    return L % q
    # return _divmod(num, den, q) % q


def _extended_gcd(a, b):
    """
    Division in integers modulus p means finding the inverse of the
    denominator modulo p and then multiplying the numerator by this
    inverse (Note: inverse of A is B such that A*B % p == 1) this can
    be computed via extended Euclidean algorithm
    http://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation
    """
    x = 0
    last_x = 1
    y = 1
    last_y = 0
    while b != 0:
        quot = a // b
        a, b = b, a % b
        x, last_x = last_x - quot * x, x
        y, last_y = last_y - quot * y, y
    return last_x, last_y


def _divmod(num, den, p):
    """Compute num / den modulo prime p

    To explain what this means, the return value will be such that
    the following is true: den * _divmod(num, den, p) % p == num
    """
    inv, _ = _extended_gcd(den, p)
    return num * inv

In [21]:
# initialization
time_start = time.time()
print("========Main LCC Starts==========")
p,q,r,g = initial(2* 10**4)

# Secret taken from the group Z_q* 
# T, N, K = 2, 10, 3
T, N, K = 7, 40, 8
secrets = [random.randint(2,q-1) for i in range(K)]
print(f'Original Secret: {secrets}')

# Phase I: Generation of shares
alphas = list(range(1, 1+N))
betas = list(range(1+N, N+K+T+1))
shares, commitments, verifications= generate_shares(N, T, K, secrets, p, q, r, g, alphas, betas)
print(f'Shares: {", ".join(str(share) for share in shares)}')
# print(f'Commitments: {", ".join(str(commitment) for commitment in commitments)}')
# print(f'verifications: {", ".join(str(verification) for verification in verifications)}')

# Phase II: Secret Reconstruction
# Picking t shares randomly for reconstruction
pool = random.sample(shares, T+K)
print(f'Combining shares: {", ".join(str(share) for share in pool)}')
secret_reconstructed = reconstruct_secret(pool, q, betas, K)
print("reconstruct_secret:",secret_reconstructed)
print(f'Original Secret: {secrets}')

time_end = time.time()
print('time cost in second:', time_end-time_start)

========Main LCC Starts==========
q = 20011

q is prime

r = 6
p = 120067

p is prime


g = 86203

Original Secret: [6711, 10761, 7484, 5678, 8541, 1241, 17620, 3886]
Shares: (1, 15989), (2, 11811), (3, 14731), (4, 8783), (5, 10843), (6, 4845), (7, 11361), (8, 14502), (9, 12174), (10, 7152), (11, 3254), (12, 13114), (13, 17659), (14, 8782), (15, 2276), (16, 2359), (17, 18315), (18, 1407), (19, 14427), (20, 7147), (21, 197), (22, 13852), (23, 18061), (24, 512), (25, 17014), (26, 2078), (27, 8664), (28, 10859), (29, 3479), (30, 17395), (31, 1700), (32, 9521), (33, 17261), (34, 1492), (35, 16950), (36, 11026), (37, 19304), (38, 4856), (39, 2072), (40, 5297)
Combining shares: (37, 19304), (19, 14427), (20, 7147), (31, 1700), (33, 17261), (21, 197), (4, 8783), (12, 13114), (11, 3254), (28, 10859), (9, 12174), (36, 11026), (6, 4845), (8, 14502), (10, 7152)
reconstruct_secret time takes: 0.0008456707000732422
reconstruct_secret: [6711, 10761, 7484, 5678, 8541, 1241, 17620, 3886]
Original Secr

In [22]:
import copy

k = T+K
n = N
p = q

F = FiniteField(p = p)
enc, dec, solveSystem = makeEncoderDecoder(n, k, p)

encoded_msg = []
for share in shares:
    x, y = share[0], share[1]
    encoded_msg.append( [F(x),F(y)] )

print(encoded_msg[0])
corrupted = copy.deepcopy(encoded_msg[:])
corrupted[0][1] = corrupted[0][1] + 1
print(corrupted[0])

Q,E = solveSystem(corrupted)
P, remainder = (Q.__divmod__(E))
print("P(x) = %r" % P)

recon_secrets = []
for i in range(K):
    recon_secrets.append(int(P(betas[i])))
print(recon_secrets)

[1 (mod 20011), 15989 (mod 20011)]
[1 (mod 20011), 15990 (mod 20011)]
P(x) = 14662 + 15870 t^1 + 1598 t^2 + 3498 t^3 + 7692 t^4 + 3532 t^5 + 15258 t^6 + 6751 t^7 + 12944 t^8 + 7446 t^9 + 11900 t^10 + 4114 t^11 + 9417 t^12 + 789 t^13 + 573 t^14
[6711, 10761, 7484, 5678, 8541, 1241, 17620, 3886]


In [23]:
def RS_reconstruct_secret(encoded_shares, Field, solveSystem, betas, K):
    start = time.time()
    encoded_msg = []
    for share in encoded_shares:
        x, y = share[0], share[1]
        encoded_msg.append( [Field(x),Field(y)] )
    Q,E = solveSystem(encoded_msg)
    P, remainder = (Q.__divmod__(E))
    
    recon_secrets = []
    for i in range(K):
        recon_secrets.append(int(P(betas[i])))
    print("reconstruct_secret time takes:", time.time() - start)
    return recon_secrets

RS_reconstruct_secret(shares, F, solveSystem, betas, K)

reconstruct_secret time takes: 0.1775674819946289


[6711, 10761, 7484, 5678, 8541, 1241, 17620, 3886]